## FastINPUT

Набор функций для формирования входного файла для расчета свойств мономолекулярных могнитов на основе диспрозия.

На вход поступает файл .xyz формата с коодинатами атомов молекулярного магнита, возвращается файл формата .input в папку inputs

Файл настроен на следующие параметры расчета

| Элемент | Базис |
| --- | --- |
|  Dy | ANO-RCC-VTZP |
|  O | ANO-RCC-VTZP |
|  N | ANO-RCC-VTZP |
|  Cl | ANO-RCC-VTZP |
|  Ho | ANO-RCC-VTZP |
|  C | ANO-RCC-VDZP |
|  S | ANO-RCC-MB |
|  H | ANO-RCC-MB |
|  F | ANO-RCC-MB |

RASSCF
| Параметр расчета | Значение |
| --- | --- |
| SYMM | 1 |
| CIMX | 200 |
| CHARGE | 0 |
| CIROOT | 21 |
| NACTEL | 9 |
| RAS 2 | 7 |
| ORBL | NOCORE |
| ORBA | COMPACT |


In [5]:
def read_xyz_file(xyz_filename):
    """Читает файл XYZ и возвращает данные атомов и их координаты."""
    with open(xyz_filename, 'r') as xyz_file:
        lines = xyz_file.readlines()
    
    # Пропускаем первые две строки (число атомов и размерность)
    atom_data = lines[2:]
    atoms = {}

    for line in atom_data:
        parts = line.split()
        if len(parts) < 4:
            continue  # Пропускаем пустые строки или некорректные данные
        atom_type = parts[0]
        coordinates = parts[1:4]

        if atom_type not in atoms:
            atoms[atom_type] = []
        atoms[atom_type].append(coordinates)

    return atoms

def get_basis_set(atom_type):
    """Возвращает базисный набор для данного типа атома."""
    basis_sets = {
        'Dy': f"{atom_type}.ANO-RCC-VTZP.",
        'O': f"{atom_type}.ANO-RCC-VTZP.",
        'C': f"{atom_type}.ANO-RCC-VDZP.",
        'S': f"{atom_type}.ANO-RCC-MB.",
        'H': f"{atom_type}.ANO-RCC-MB.",
        'F': f"{atom_type}.ANO-RCC-MB.",
        'N': f"{atom_type}.ANO-RCC-VTZP.",
        'Cl': f"{atom_type}.ANO-RCC-VTZP.",
        'Ho': f"{atom_type}.ANO-RCC-VTZP."
    }
    return basis_sets.get(atom_type)

def write_input_file(input_filename, atoms, exact_name):
    """Записывает данные в файл input на основе атомов и их координат."""
    with open(input_filename, 'w') as input_file:
        # Записываем заголовок
        input_file.write("&GATEWAY\n"
                         "Douglas-Kroll\n"
                         "\nAMFI\n"
                         "ANGM\n"
                         f" {atoms['Dy'][0][0]} {atoms['Dy'][0][1]} {atoms['Dy'][0][2]}  Angstrom\n\n")

        for atom_type, coords in atoms.items():
            basis_set = get_basis_set(atom_type)
            if basis_set is None:
                continue  # Пропускаем атомы, которые не нужны

            # Записываем базисный набор
            input_file.write("Basis Set\n")
            input_file.write(f"{basis_set}\n")
            for i, coord in enumerate(coords, start=1):
                input_file.write(f" {atom_type}{i}    {' '.join(coord)}    Angstrom\n")
            input_file.write("End of Basis Set\n\n")

        # Записываем дополнительные параметры
        input_file.write(
            '&SEWARD\n'
            'HIGH Cholesky\n\n'
            '&RASSCF\n'
            'SYMM\n'
            '1\n'
            'CIMX\n'
            '200\n'
            'charge\n'
            '0\n'
            'Spin\n'
            '6\n'
            'ciroot\n'
            '21 21 1\n'
            'Nactel\n'
            '9 0 0\n'
            'Ras2\n'
            '7\n'
            'ORBL\n'
            'nocore\n'
            'ORBA\n'
            'compact\n'
            '>>> COPY $Project.JobIph JOB001\n\n'
            '&RASSI\n'
            'spinorbit\n'
            'nrofjobiphs=1 21\n'
            '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21\n'
            'mees\n'
            'PROP\n'
            '3\n'
            '\'ANGMOM\' 1\n'
            '\'ANGMOM\' 2\n'
            '\'ANGMOM\' 3\n\n'
            '&SINGLE_ANISO\n'
            'MLTP\n'
            '8\n'
            '2 2 2 2 2 2 2 2\n'
            'CRYS\n'
            'Dy\n'
            'XFIE\n'
            '0.5\n'
            'TINT\n'
            '0 300 301\n'
            'HINT\n'
            '0 5 51\n'
            'TMAG\n'
            '3 2.0 4.0 6.0\n'
            'UBAR\n\n'
            f'>>> COPY $Project.aniso $CurrDir/{exact_name}.aniso\n'
        )

def xyz_to_input(xyz_filename, input_filename, exact_name):
    """Основная функция для преобразования файла XYZ в файл input."""
    atoms = read_xyz_file(xyz_filename)
    write_input_file(input_filename, atoms, exact_name)

# Использование
exact_name = 'artion1new'
xyz_filename = f'./master/structures/{exact_name}.xyz'
xyz_to_input(xyz_filename, f'./master/inputs/{exact_name}.input', exact_name)